In [31]:
# Tentativa de reformular graph_utils, colocando necessidade de segregar a malha ou não, e identificação de sentido ou não.
import pandas as pd
import sqlalchemy
from itertools import pairwise
from routes import routes
from utils import graph_utils, station_manager, visualization_utils
import networkx as nx

In [32]:
# import stations
stations = routes.extracts('../data/processed/stations.gpkg')
remover = routes.select_criteria(stations) # filter stations (same criteria as routes)
stations = station_manager.remove_stations(stations, remover) # remover as estações deletadas; aqui tem que ser o mesmo critério de routes
# make graph
G = graph_utils.generate_graph(stations)

In [33]:
engine = sqlalchemy.create_engine('sqlite:///../data/processed/database.db')

def import_query(file_path):
    with open(file_path, 'r') as file:
        return file.read()

query = """SELECT 
    o.data,
    o.id,
    o.TU,
    o.ValorTKU,
    f.ferrovia,
    f.cliente,
    f.mercadoria,
    f.origem,
    f.destino,
    f.dist_media,
    f.intermed,
    f.len_Dijkstra,
    f.rota
FROM 
    ocorrencias o
LEFT JOIN 
    fluxos_rotas_2017 f
ON 
    o.id = f.id
WHERE 
    o.data >= '2023-01-01' and o.data <= '2024-12-01';
"""
fluxos = pd.read_sql(query, engine)

In [51]:
def get_entrepatios(train, rail, sentido=False):
    main_list = train.split()
    main_list_ep = list(pairwise(main_list))
    rail_ep = list(pairwise(rail))

    if sentido:
        #print("Flag ativada: o sentido de rail vai ser levado em consideração.")
        return [item for item in main_list_ep if item in rail_ep] # interseçao

    else: # sentido is False
        crescente = list(pairwise(rail))
        decrescente = list(pairwise(rail[::-1]))

        intersec_crescente = [item for item in main_list_ep if item in crescente]
        intersec_decrescente = [item for item in main_list_ep if item in decrescente]

        return intersec_crescente + intersec_decrescente # união das duas interseccoes
    
def find_edge(G, entrepatio, operator):
    try:
        if G.edges[entrepatio]['ferrovia']==operator:
            return G.edges[entrepatio]['weight']
        else:
            return 0
    except:
        return 0

def train_on_rail(G, train, rail, tu, operator, valor_retorno='tku', sentido=False):
    km = 0
    entrepatios = get_entrepatios(train, rail, sentido)

    for par in entrepatios:
        if find_edge(G, par, operator) is None:
            print('Aresta não encontrada: ' + par + ' de ' + operator)

        km += find_edge(G, par, operator)
    
    if valor_retorno == "dist":
        return km
    elif valor_retorno == "tku":
        return km * tu
    elif valor_retorno == "bool":
        return bool(km)
    elif valor_retorno == "tu":
        return bool(km)*tu
    else:
        raise ValueError("Opção inválida para 'valor_retorno'. Use 'dist', 'tku' ou 'bool' ou 'tu'.")

In [59]:
# teste unitário
path = stations[stations['ferrovia']=='EFVM']['estacao'].tolist()
train_on_rail(G, 'VTU V04 V03 VAB VFN VAR VPA VCL VIA VMC VBG VAY VIU VRD VCN VCP VBC VST VTR VGV VPC VFS VJC VIC',
    path, 4446.0, 'EFVM', valor_retorno='tku', sentido=False)

1925611.5060000003

In [57]:
stations[stations['ferrovia']=='EFVM'].head(50)

,id,ferrovia,linha,estacao,NomeEstacao,sequencia,extensao,terminal,intercambio,points_geometry,geometry
286,5482,EFVM,Desemb. Drumond - Piçarrão,VDD,Desembargador Drumond,1,0.000,1,0,POINT (-43.018132 -19.73901),None
287,5483,EFVM,Desemb. Drumond - Piçarrão,VPI,Picarrao,3,16.100,0,0,None,None
801,7067,EFVM,Pedro Nolasco - Porto Velho,VPN,Pedro Nolasco,1,0.000,1,1,POINT (-40.355345 -20.327886),None
802,7068,EFVM,Pedro Nolasco - Porto Velho,VPV,Vitória/Porto Velho,2,3.000,1,1,POINT (-40.347017 -20.328425),"LINESTRING (-40.34479 -20.32729, -40.34514 -20..."
803,5514,EFVM,Piraqueaçu - Aracruz,VPA,Piraqueaçu,1,0.000,1,0,POINT (-40.366438 -19.748594),None
804,5515,EFVM,Piraqueaçu - Aracruz,VAZ,Aracruz,2,47.000,1,0,POINT (-40.067966 -19.82525),"LINESTRING (-40.36176 -19.72246, -40.36128 -19..."
946,5486,EFVM,Ramal Conceição,VBF,Ent. km 540,1,0.000,0,0,POINT (-43.224459 -19.632753),None
947,5487,EFVM,Ramal Conceição,VCE,Conceição,2,5.607,1,0,POINT (-43.265849 -19.65123),"LINESTRING (-43.22446 -19.63275, -43.22616 -19..."
999,5484,EFVM,Ramal João Paulo,VBF,Ent. km 540,1,0.000,0,0,POINT (-43.224459 -19.632753),None
1000,5485,EFVM,Ramal João Paulo,VJP,João Paulo,2,4.170,1,0,POINT (-43.222764 -19.608971),"LINESTRING (-43.22447 -19.63275, -43.22613 -19..."


In [43]:
# apenas fluxos com data entre 2023-01-01 e 2024-12-01
dist, path = graph_utils.must_pass(G, ['VTU','VIC'])
path

['VTU',
 'V04',
 'V03',
 'VAB',
 'VFN',
 'VAR',
 'VPA',
 'VCL',
 'VIA',
 'VMC',
 'VBG',
 'VAY',
 'VIU',
 'VRD',
 'VCN',
 'VCP',
 'VBC',
 'VST',
 'VTR',
 'VGV',
 'VPC',
 'VFS',
 'VJC',
 'VIC']

In [45]:
fluxos['find_tku'] = fluxos.apply(lambda x: graph_utils.find_tku(G, x['rota'], path, x['TU'], 'EFVM'), axis=1)

In [46]:
fluxos['train_on_rail'] = fluxos.apply(lambda x: graph_utils.train_on_rail(G, x['rota'], path, x['TU'], 'EFVM',
                                                                valor_retorno='tku', sentido=False), axis=1)


In [47]:
fluxos['Diff'] = fluxos['find_tku'] - fluxos['train_on_rail']

In [48]:
fluxos.sort_values(by='Diff', ascending=False)

,data,id,TU,ValorTKU,ferrovia,cliente,mercadoria,origem,destino,dist_media,intermed,len_Dijkstra,rota,find_tku,train_on_rail,Diff
52724,2024-11-01 00:00:00.000000,77356RMC,23.0,20877.0,RMC,LOUIS DREYFUS COMMODITIES BRASIL S/A,Açúcar,PIT,ICZ,907.705,PIT ZRL ZTO ZOI ZIQ ZBV ZQB ZKE ZEV ZPG IAA IP...,907.705,PIT POU PGO ZRL ZFN ZMR ZGI ZVP ZZM ZKY ZEC ZT...,0.000,0.000,0.0
0,2023-01-01 00:00:00.000000,H4495EFVM,10.0,6708.0,EFVM,GERDAU ACOMINAS S/A,Prd. Siderúrgicos - Outros,VOB,VTU,670.876,VOB ELF VDD V03 VTU,678.402,VOB ELF VCB VFU VRV VTO VAL VFZ VBT VAG VCS OF...,4331.110,4331.110,0.0
1,2023-01-01 00:00:00.000000,27873RMN,231.0,380455.0,RMN,AMAGGI EXPORTACAO E IMPORTACAO LTDA,Grãos - Milho,TRO,ICZ,1646.994,TRO TMI ZTO ZOI ZIQ ZBV ZQB ZKE ZEV ZPG IAA IP...,1650.861,TRO TSF TAL TBS TSA TIQ TCT TMJ TEP TBE TAG TB...,0.000,0.000,0.0
2,2023-01-01 00:00:00.000000,18386RMS,786.0,622070.0,RMS,CARGILL AGRICOLA S A,Grãos - Trigo,NIJ,NRG,791.438,NIJ NCZ NTM NCY NBG NRG,791.438,NIJ NAN NFM NCZ NBN NEP NTP NJC NGU NVS NPI NF...,0.000,0.000,0.0
3,2023-01-01 00:00:00.000000,72007RMS,2738.0,2020485.0,RMS,NOBLE BRASIL,Grãos - Trigo,NCZ,NRG,737.942,NCZ NTM NCY NBG NRG,737.942,NCZ NBN NEP NTP NJC NGU NVS NPI NFP NTM NSM NB...,0.000,0.000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24,2023-01-01 00:00:00.000000,I0362FTL,80.0,48960.0,FTL,FERROVIA TRANSNORDESTINA LOGÍSTICA S.A,Demais Produtos,ATO,BBV,612.001,ATO ALT BBV,610.841,ATO ARS AEA AHP ACN ANJ AXK APM AJI AOA AXA AC...,0.000,0.000,0.0
25,2023-01-01 00:00:00.000000,I2866EFVM,7401.0,4011904.0,EFVM,COMPANHIA COREANO BRASILEIRA DE PELOTIZAÇÃO - ...,Minério de Ferro,VJP,VTU,542.076,VJP VBF V03 VTU,542.021,VJP VBF VLB V77 VDD DDC VAD VAC VMR VIC VJC VF...,3205454.511,3205454.511,0.0
26,2023-01-01 00:00:00.000000,I27899EFC,138039.0,130638177.0,EFC,VALE S/A,Minério de Ferro,QSS,QPM,946.386,QSS QRS QPM,958.037,QSS QRS QSL QMA QAL QNV QSI QRO QPM,0.000,0.000,0.0
27,2023-01-01 00:00:00.000000,86581RMS,948.0,1377147.0,RMS,IPIRANGA PRODUTOS DE PETROLEO S.A.,Álcool,ZOU,NPY,1452.687,ZOU LAP LUS LEB LRI LLS NRO NGL NPY,1452.687,ZOU LMQ LCM LIG LBD LLJ LSN LCP LGH LUR LJY LI...,0.000,0.000,0.0


In [54]:
fluxos[fluxos['id']=='H8092EFVM']#.rota.unique()

array(['VTU V04 V03 VAB VFN VAR VPA VCL VIA VMC VBG VAY VIU VRD VCN VCP VBC VST VTR VGV VPC VFS VJC VIC'],
      dtype=object)